In [43]:
import sys

In [44]:
# Finder finds a loader

In [45]:
module_name = 'matplotlib.pyplot'
loader = None
for finder in sys.meta_path:
    if not loader:
        loader = finder.find_module(module_name)
        print(loader)

None
None
None
None
None
None


In [46]:
loader

In [47]:
print(loader)

None


In [48]:
sys.modules['matplotlib']

<module 'matplotlib' from '/home/sasidhar/anaconda36/lib/python3.6/site-packages/matplotlib/__init__.py'>

In [49]:
m = loader.load_module()

AttributeError: 'NoneType' object has no attribute 'load_module'

In [50]:
m

<module 'numpy' from '/home/sasidhar/anaconda36/lib/python3.6/site-packages/numpy/__init__.py'>

In [51]:
os = m

In [52]:
sys.modules['os']

<module 'os' from '/home/sasidhar/anaconda36/lib/python3.6/os.py'>